# Imports

In [1]:
import itertools
import math

from joblib import Parallel, delayed
import khmer
import numpy as np
import pandas as pd
import screed
import seaborn as sns
import sencha
from tqdm import tqdm


%load_ext autoreload
%autoreload 2




In [2]:
sencha.__file__

'/home/olga/code/sencha/sencha/__init__.py'

Files from https://osf.io/f8eq3/files/

In [3]:
uniprot_folder = '/home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded'
! ls -lha $uniprot_folder/uniprot*.fasta.gz

-rw-r--r-- 1 olga czb  78M Jun  4 17:06 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot-reviewed_yes__NOT_taxonomy__Eukaryota_9EUKA_2759.fasta.gz
-rw-r--r-- 1 olga czb  55M Feb 20 16:06 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot-reviewed_yes+taxonomy_2759.fasta.gz
-rw-r--r-- 1 olga czb 4.1M Jun  4 17:06 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot-reviewed_yes__taxonomy_Archea_2157.fasta.gz
-rw-r--r-- 1 olga czb  69M Jun  4 17:06 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot-reviewed_yes__taxonomy__Bacteria_9BACT.fasta.gz
-rw-r--r-- 1 olga czb  22M May 23 10:21 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot-reviewed_yes+taxonomy_Mammalia_9MAMM_40674.fasta.gz
-rw-r--r-- 1 olga czb  43M May 23 10:17 /home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manu

In [4]:
# 
# ! ls -1 $uniprot_folder/uniprot_sprot*

In [5]:
PROTEIN_SIGMA = 20

def get_equivalent_protein_ksize(protein_ksize, sigma):
    return protein_ksize * math.log(PROTEIN_SIGMA) / math.log(sigma)

get_equivalent_protein_ksize(protein_ksize=5, sigma=6)

8.359750080865052

In [6]:
for i in range(2, 20):
    k = get_equivalent_protein_ksize(protein_ksize=9, sigma=i)
    print(f'{i}\t{k:.2f}')

2	38.90
3	24.54
4	19.45
5	16.75
6	15.05
7	13.86
8	12.97
9	12.27
10	11.71
11	11.24
12	10.85
13	10.51
14	10.22
15	9.96
16	9.72
17	9.52
18	9.33
19	9.16


In [7]:
get_equivalent_protein_ksize(protein_ksize=9, sigma=6)

15.047550145557093

In [8]:
get_equivalent_protein_ksize(protein_ksize=9, sigma=2)

38.89735285398626

In [9]:
get_equivalent_protein_ksize(protein_ksize=9, sigma=8)

12.965784284662087

In [10]:
get_equivalent_protein_ksize(protein_ksize=12, sigma=6)

20.063400194076124

In [11]:
get_equivalent_protein_ksize(protein_ksize=5, sigma=2)

21.609640474436812

In [12]:
get_equivalent_protein_ksize(protein_ksize=14, sigma=2)

60.50699332842307

# Define fastas and ksizes

In [13]:
mammalia_fasta = f'{uniprot_folder}/uniprot-reviewed_yes+taxonomy_Mammalia_9MAMM_40674.fasta.gz'
eukaryota_fasta = f'{uniprot_folder}/uniprot-reviewed_yes+taxonomy_2759.fasta.gz'
opsithokonta_fasta = f'{uniprot_folder}/uniprot-reviewed_yes+taxonomy_Opisthokonta_33154.fasta.gz'
swissprot_fasta = f'{uniprot_folder}/uniprot_sprot.fasta.gz'
not_eukaryota_fasta = f'{uniprot_folder}/uniprot-reviewed_yes__NOT_taxonomy__Eukaryota_9EUKA_2759.fasta.gz'
bacteria_fasta = f'{uniprot_folder}/uniprot-reviewed_yes__taxonomy__Bacteria_9BACT.fasta.gz'
archea_fasta = f'{uniprot_folder}/uniprot-reviewed_yes__taxonomy_Archea_2157.fasta.gz'
uniref_fasta = '/home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/uniref/uniref100/uniref100.fasta.gz'

fastas = {
    "mammalia": mammalia_fasta,
    "eukaryota": eukaryota_fasta,
    "opisthokonta": opsithokonta_fasta,
    "swissprot": swissprot_fasta,
    "not_eukaryota": not_eukaryota_fasta,
    "bacteria": bacteria_fasta,
    "archea": archea_fasta,
#     "uniref": uniref_fasta,
}


In [14]:
from sencha.index import maybe_save_peptide_index, make_peptide_index, ALPHABET_SIZES

## Make an index with just mammalian sequences for checking/debugging

In [15]:
mammal_index = make_peptide_index(mammalia_fasta, peptide_ksize=9, molecule='protein', debug=False)

khmer.calc_expected_collisions(mammal_index)

mammal_index.n_unique_kmers()


index_filename = 'mammal_index__ksize-9__molecule-protein.bloomfilter'
mammal_index.save(index_filename)

mammal_index_loaded = khmer.Nodegraph.load(index_filename)
mammal_index_loaded.n_unique_kmers()

67241it [01:03, 1055.94it/s]


0

## Count number of sequences and total length of sequences per database

In [16]:
# lengths = {}
# n_sequences = {}

# for name, fasta in fastas.items():
#     print(name)
#     length = 0
#     n_seqs = 0
#     with screed.open(fasta) as records:
#         for record in tqdm(records):
#             length += len(record['sequence'])
#             n_seqs += 1
#     n_sequences[name] = n_seqs 
#     lengths[name] = length

### Make dataframe and write to file

In [17]:
# fasta_sizes = pd.DataFrame({'n_sequences': n_sequences, 'n_residues': lengths})
# fasta_sizes['n_residues_log10'] = np.log10(fasta_sizes['n_residues'])
# fasta_sizes['n_sequences_log10'] = np.log10(fasta_sizes['n_sequences'])
# fasta_sizes['tablesize_log10'] = np.ceil(fasta_sizes['n_residues_log10']).astype(int)
# fasta_sizes['tablesize'] = np.power(10, fasta_sizes['tablesize_log10'])
# fasta_sizes.to_csv('total_sequence_lengths.csv')
# fasta_sizes

In [18]:
fasta_sizes = pd.read_csv('total_sequence_lengths.csv', index_col=0)
print(fasta_sizes.shape)
fasta_sizes.head()

(8, 5)


,n_sequences,n_residues,n_residues_log10,n_sequences_log10,tablesize
mammalia,67241,33332118,7.522863,4.827634,8
eukaryota,190642,83870914,7.923611,5.280219,8
opisthokonta,141854,64718973,7.811032,5.151842,8
swissprot,561911,202173710,8.305725,5.749668,9
not_eukaryota,371102,118211069,8.072658,5.569493,9


# Make a bunch of sencha indices, compute number of unique k-mers

In [19]:
# fastas_minus_all = {'mammalia': mammalia_fasta, 'eukaryota': eukaryota_fasta, 
#           'opisthokonta': opsithokonta_fasta         }


In [20]:


lines = []

def create_index_and_compute_stats(name, fasta, alphabet, ksize, tablesize=int(1e8)):
    index = make_peptide_index(fasta, ksize, alphabet, tablesize=tablesize, force=True)
    expected_collisions = khmer.calc_expected_collisions(index)
    maybe_save_peptide_index(fasta, index, alphabet, save_peptide_index=True)

    n_unique_kmers = index.n_unique_kmers()
    sigma = ALPHABET_SIZES[alphabet]
    line = [name, alphabet, ksize, sigma, expected_collisions, n_unique_kmers]
    del index
    return line

In [21]:
N_JOBS = 32


def format_index_stats(lines):
    columns = ['name', 'molecule', 'ksize', 'sigma', 'expected_collisions', 'n_unique_kmers']
    df = pd.DataFrame(lines, columns=columns)
    df['n_theoretical_kmers_log10'] = df['ksize'] * np.log10(df['sigma'])
    df['n_unique_kmers_log10'] = np.log10(df['n_unique_kmers'])
    df['unique_over_theoretical_log10'] = df['n_unique_kmers_log10'] - df['n_theoretical_kmers_log10']
    return df

In [ ]:
dfs = []

for name, fasta in fastas.items():
    print(f'name: {name}')
    iterator = itertools.product(['protein20', 'dayhoff6', 'hp2'], range(2, 51))
    tablesize = 10 ** fasta_sizes.loc[name, 'tablesize']
    this_fasta_lines = Parallel(n_jobs=N_JOBS, verbose=True)(
        delayed(create_index_and_compute_stats)(
            name, fasta, alphabet, ksize, tablesize=tablesize) for alphabet, ksize in iterator)
    df = format_index_stats(this_fasta_lines)
    df.to_csv(f'n_observed_vs_theoretical_kmers_{name}.csv', index=False)
    dfs.append(df)

index_stats = pd.concat(dfs)
# index_stats['n_theoretical_kmers'] = np.power(index_stats.sigma, index_stats.ksize)
# index_stats['unique_over_theoretical'] = index_stats.n_unique_kmers / index_stats.n_theoretical_kmers
# index_stats['unique_over_theoretical_log10'] = -1 * np.log10(index_stats['unique_over_theoretical'])
index_stats.to_csv('n_observed_vs_theoretical_kmers_per_uniprot_database.csv', index=False)
print(index_stats.shape)
index_stats.head()

name: mammalia


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 147 out of 147 | elapsed:  6.8min finished
[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


name: eukaryota


[Parallel(n_jobs=32)]: Done 147 out of 147 | elapsed: 14.8min finished
[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


name: opisthokonta


[Parallel(n_jobs=32)]: Done 147 out of 147 | elapsed: 11.5min finished
[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


name: swissprot


In [ ]:
# df = format_index_stats(this_fasta_lines)
# df.to_csv(f'n_observed_vs_theoretical_kmers_{name}.csv', index=False)
# dfs.append(df)

In [ ]:
df

In [ ]:
# columns = ['name', 'molecule', 'ksize', 'sigma', 'expected_collisions', 'n_unique_kmers']
# index_stats = pd.DataFrame(itertools.chain(*lines), 
#                            columns=columns)

# index_stats['n_unique_kmers_log10'] = np.log10(index_stats.n_unique_kmers,)
# # index_stats['n_theoretical_kmers'] = np.power(index_stats.sigma, index_stats.ksize)
# index_stats['n_theoretical_kmers_log10'] = index_stats.ksize * np.log10(index_stats.sigma)
# # index_stats['unique_over_theoretical'] = index_stats.n_unique_kmers / index_stats.n_theoretical_kmers
# index_stats['unique_over_theoretical_log10'] = index_stats['n_unique_kmers_log10'] - index_stats['n_theoretical_kmers_log10']
print(index_stats.shape)
index_stats.head()

## Manually do swissprot in serial to find the bug

In [ ]:
name

In [ ]:
fasta

In [ ]:
# import tqdm

# def nop(it, *a, **k):
#     return it

# tqdm.tqdm = nop

In [ ]:


this_fasta_lines  = []
# print(f'name: {name}')
iterator = itertools.product(['protein20', 'dayhoff6', 'hp2'], range(2, 32))
for alphabet, ksize in iterator:
    line = create_index_and_compute_stats(name, fasta, alphabet, ksize, tablesize=1e10)
    this_fasta_lines.append(line)
# this_fasta_lines = Parallel(n_jobs=32, verbose=True)(
#     delayed(create_index_and_compute_stats)(
#         name, fasta, alphabet, ksize) for alphabet, ksize in iterator)
# lines.append(this_fasta_lines)

In [ ]:
alphabet

In [ ]:
ksize

In [ ]:
lines

In [ ]:
lines 

In [ ]:
index_stats.tail()

In [ ]:
index_stats = pd.DataFrame(itertools.chain(*lines), columns=['name', 'molecule', 'ksize', 'sigma', 'expected_collisions', 'n_unique_kmers'])
print(index_stats.shape)
index_stats.head()

In [ ]:
index_stats['n_theoretical_kmers'] = np.power(index_stats.sigma, index_stats.ksize)
index_stats.head()

In [ ]:
index_stats['unique_over_theoretical'] = index_stats.n_unique_kmers / index_stats.n_theoretical_kmers
index_stats.head()

In [ ]:
index_stats.tail()

In [ ]:
index_stats['unique_over_theoretical_log10'] = -1 * np.log10(index_stats['unique_over_theoretical'])
index_stats.head()

In [ ]:
index_stats.to_csv('n_observed_vs_theoretical_kmers_per_uniprot_database.csv', index=False)

In [ ]:
sns.catplot(x='ksize', y='n_unique_kmers', 
            data=index_stats, hue='name', col='molecule')

In [ ]:
sns.catplot(x='ksize', y='n_unique_kmers', 
            data=index_stats, hue='name', col='molecule')

In [ ]:
mammalia_fasta